<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CST Template Notebook</b> <br>
Contact author(s): <i>Christina Williams, Leanne Guy</i> <br>
Last verified to run: <i>2023-09-12</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

**Description:** _Demonstration of how to use the cutout tool for calexp and deepCoadds_

## 1. Introduction

This is a brief demonstration of how to use the cutout tool. The calls to the cutout tool are embedded in the `make_image_cutout` function.

### 1.1 Package Imports


In [ ]:
# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

# Import general python packages
import numpy as np
import pandas
from pandas.testing import assert_frame_equal
import uuid
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
from lsst.afw.image import Image, ImageF
from lsst.afw.image import Exposure, ExposureF

# Plotting with MPL
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service

# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Astropy
from astropy import units as u


### 1.2 Define Functions and Parameters

Two functions are provided here. `make_image_cutout` provides the cutout service for either calexp or deepCoadds, and `plotImage` is simply for convenience of plotting the output.

In [ ]:
def plotImage(exposure: ExposureF, img_opt: dict = None):
    """Plot and image using matplotlib
   
   Parameters
    ----------
    image : `Exposure`
        the image to plot
        
    opts : ``
   
   Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    
    fig, ax = plt.subplots()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(exposure.image)
    plt.show()


In [ ]:
def make_image_cutout(tap_service, ra, dec, cutout_size=0.01, 
                      imtype=None, dataId=None, filename=None): 
    """Make a cutout using the cutout tool
   
   Parameters
    ----------
    # cutout_size is in degrees
    # if imtype None, assumes deepCoadd. dataId must correspond to that for provided imtype
    # for deepCoadd, dataId should have format {'band':'band', 'tract':tract, 'patch':patch} (band optional)
    # for calexp, dataId should have format {'visit':visitId, 'detector':detector}
   Returns
    -------
    
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    
    if imtype == 'calexp':
        
        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_visit, lsst_detector, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.calexp' " + \
            "AND lsst_visit = " + str(dataId["visit"]) + " " + \
            "AND lsst_detector = " + str(dataId["detector"])
        results = tap_service.search(query) # THIS IS TAP SERVICE

    else:
        # Find the tract and patch that contain this point
        # this is how its done using butler:
        tract = dataId["tract"]
        patch = dataId["patch"]
        
        # add optional default band if it is not contained in the dataId
        if 'band' in dataId:
            band = dataId["band"]
        else:
            band = 'z'

        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_tract = " + str(tract) + " " + \
            "AND lsst_patch = " + str(patch) + " " + \
            "AND lsst_band = " + "'" + str(band) + "' "
        results = tap_service.search(query) 

    #Get datalink
    dataLinkUrl = results[0].getdataurl()
    f"Datalink link service url: {dataLinkUrl}"
    auth_session = service._session
    dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
    f"{dl_results.status}"

    # from_resource: creates a instance from a number of records and a Datalink Resource.
    sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
                 spherePoint.getDec().asDegrees()*u.deg, 
                 cutout_size* u.deg)

    if filename:
        sodaCutout = 'cutouts/'+filename
        
    else:
        sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')

    with open(sodaCutout, 'bw') as f:
        f.write(sq.execute_stream().read())
        
    #sodaCutout is just the filename...
    return sodaCutout


## 2. Generating an image cutout using the cutout tool

First, demonstrate a simple example retrieving a cutout of a deepCoadd.

In [ ]:
# Set afw display backend to matplotlib
afwDisplay.setDefaultBackend('matplotlib')

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

In [ ]:
# use this to query for the deepCoadd's patch and tract
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry


In [ ]:
# this will be fed to make_image_cutout to allow cutout tool to find appropriate info
service = get_tap_service("tap")

In [ ]:
# galaxy cluster:
#ra = 55.7467 
#dec = -32.2862

# random DIAobject 
ra = 72.5383603
dec = -44.4248533

# Define a spatial point
spherePoint = lsst.geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)

# Find the tract and patch that contain this point
skymap = butler.get('skyMap') # skymap is used for coadd projection
tract = skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)

# Now create a dataId from the tract, patch and filter information
dataId = {'band':'i', 'tract':tract.tract_id, 'patch': patch.getSequentialIndex()}



### 2.1 Call the function "make_image_cutout" defined above on a deepCoadd data ID

In [ ]:
imtype = 'deepCoadd' # this is the default, imtype=None will assume deepCoadd

test = make_image_cutout(service, ra, dec, dataId=dataId, imtype=imtype, cutout_size=0.005)
plotImage(ExposureF(test))


### 2.2 Example of cutout for calexp files, using a diaObject


In [ ]:
diaObjectId = 1253478440036730088

ccdquery = "SELECT TOP 10 dia.coord_ra, dia.coord_dec, " + \
                        "dia.diaObjectId,  dia.ccdVisitId, dia.band, " + \
                        "cv.visitId, cv.physical_filter, cv.detector, cv.obsStartMJD, " + \
                         "cv.expMidptMJD " + \
                         "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject as dia " + \
                         "JOIN dp02_dc2_catalogs.CcdVisit as cv ON cv.ccdVisitId = dia.ccdVisitId " + \
                         "WHERE dia.diaObjectId = "+str(diaObjectId)+" AND cv.band = 'i'"
    
diaccdsearch = service.search(ccdquery)
forcedSrc = diaccdsearch.to_table()
del diaccdsearch

In [ ]:
wh = np.argsort(forcedSrc['obsStartMJD'])
forcedSrc[wh]

In [ ]:
# This cell plots the cutouts for calexps returned by the search above, in order of MJD 
nplots = 1
for i in range(nplots):
    dataId_calexp = {'visit':forcedSrc['visitId'][wh][i], 'detector':forcedSrc['detector'][wh][i]}
    test = make_image_cutout(service, ra, dec, cutout_size=0.005,imtype='calexp', dataId=dataId_calexp,
                             filename='cutout_'+str(i)+'.fits')

    plotImage(ExposureF(test))


